# Worksheet 08

Name:  Kevin Smith
UID: U76047763

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [8]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[2.221336632737459, 5.0686924652488115, 5.81137391937799, 2.2190389807313506, 4.7821498668621905, 4.70136881478748, 3.9144919240128226, 4.473451877203494, 5.208661820383679, 4.896408974025199]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [9]:
c2 = np.random.normal(8,1,10).tolist()
print(c2)

[6.471284736489576, 8.748968542742755, 8.14474456272164, 8.839855241606825, 8.168418968259974, 6.648249430315453, 10.219827523751883, 7.65765718553343, 9.966806075495771, 9.346149614212907]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [10]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[9.346149614212907, 4.896408974025199, 5.208661820383679, 9.966806075495771, 7.65765718553343, 4.473451877203494, 3.9144919240128226, 10.219827523751883, 4.70136881478748, 4.7821498668621905]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Prior probability: $[0.5,0.5]$

Gaussian distribution 1: mean $5$, variance $1$

Gaussian distribution 2: mean $8$, variance $1$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [18]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.896408974025199, 5.208661820383679, 4.473451877203494, 3.9144919240128226, 4.70136881478748, 4.7821498668621905]
[9.346149614212907, 9.966806075495771, 7.65765718553343, 10.219827523751883]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 4.662755546212478,  mean_2 = 9.297610099748498
var_1 = 0.16068131670486344,  var_2 = 22.47940420153326


The means are fairly close, but could be closer. mean_1 is about 0.34 off, while mean_2 is about 1.3 off.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [19]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  9.346149614212907
probability of observing that point if it came from cluster 0 =  8.256312638067232e-185
probability of observing that point if it came from cluster 1 =  0.01774697170737011
point =  4.896408974025199
probability of observing that point if it came from cluster 0 =  0.8625427859877188
probability of observing that point if it came from cluster 1 =  0.01741010393008851
point =  5.208661820383679
probability of observing that point if it came from cluster 0 =  0.00773554242102416
probability of observing that point if it came from cluster 1 =  0.01745583309250687
point =  9.966806075495771
probability of observing that point if it came from cluster 0 =  6.050699271098114e-237
probability of observing that point if it came from cluster 1 =  0.017739151024752525
point =  7.65765718553343
probability of observing that point if it came from cluster 0 =  9.061560197864862e-76
probability of observing that point if it came from cluster 1 =  0.01769984907126309
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [24]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5032131172378504,  P(C_2) = 0.4350567915655885
mean_1 = 3.178282019866314,  mean_2 = 7.846081229951805
var_1 = 2.364342967127477,  var_2 = 38.443703618334254


These calculated means are closer! mean_1 is closer by about 0.1 while mean_2 is closer by around 1.1

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [25]:
prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

9.346149614212907
Probability of coming from C_1 = 6.978356173271929e-183
Probability of coming from C_2 = 1.0

4.896408974025199
Probability of coming from C_1 = 0.9867222532735375
Probability of coming from C_2 = 0.013277746726462489

5.208661820383679
Probability of coming from C_1 = 0.39929987420917035
Probability of coming from C_2 = 0.6007001257908297

9.966806075495771
Probability of coming from C_1 = 5.116394180298033e-235
Probability of coming from C_2 = 1.0

7.65765718553343
Probability of coming from C_1 = 7.679353785488138e-74
Probability of coming from C_2 = 1.0

4.473451877203494
Probability of coming from C_1 = 0.990764551053337
Probability of coming from C_2 = 0.009235448946662887

3.9144919240128226
Probability of coming from C_1 = 0.00420232307096567
Probability of coming from C_2 = 0.9957976769290343

10.219827523751883
Probability of coming from C_1 = 3.946164957012985e-258
Probability of coming from C_2 = 1.0

4.70136881478748
Probability of coming from C_1 = 0.995

They're the same! Might be an issue with my code but they are giving the same values.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [27]:
probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    if p[1] > p[2]:
        print("This point is most likely from C_1")
    elif p[2] > p[1]:
        print("This point is most likely from C_2")
    else:
        print("This point is equally likely to have come from C_1 and C_2")

9.346149614212907
This point is most likely from C_2
4.896408974025199
This point is most likely from C_1
5.208661820383679
This point is most likely from C_2
9.966806075495771
This point is most likely from C_2
7.65765718553343
This point is most likely from C_2
4.473451877203494
This point is most likely from C_1
3.9144919240128226
This point is most likely from C_2
10.219827523751883
This point is most likely from C_2
4.70136881478748
This point is most likely from C_1
4.7821498668621905
This point is most likely from C_1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      YES       |
| A  C |      NO       |
| A  D |      NO       |
| A  E |      NO       |
| B  C |      YES       |
| B  D |      NO       |
| B  E |      NO       |
| C  D |      NO       |
| C  E |      NO       |
| D  E |      YES       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

$\frac{n(n-1)}{2}$

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

1 agreement for cluster 1 (DE), and 6 agreements for cluster 2 (FG, FH, FI, GH, GI, and HI)

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

We will represent Clusterings via a 2D matrix $A$ where each row represents one Datapoint, and each column represents a Clustering. Elements will be numbers representing assigned cluster ID.

For example, column 1 might look like $[3,4,4,2,3,3,1,2,1,1,4,1,3,2]$ — this represents Clustering 1's cluster ID assignments for each of the points.

Row 5 might look like $[1,1,4,2,3,2,5,1,7,3]$ — representing the Datapoint 5's cluster ID assignments according to each of the Clusterings.

ALGORITHM:

    A = 2D matrix as described above.
    
    disagreementTotal = 0

    for i in range(len(A[0]-1)):                   #for each column (except the last)...
        for j in range(i+1,len(A[0])):             #for each remaining column...
            for k in range(numClusters(i)):
                clusterID = k+1
                iterate through each possible pair of points within this clusterID under column i
                if that pair of points has conflicting clusterIDs in column j:
                    disagreementTotal += 1
    
    return disagreementTotal
            
            